In [1]:
import os
import sys
code_path = r"D:\DATN\QA_System\code"
sys.path.append(code_path)

# Import trực tiếp từ các file
from utils import convert_df_to_documents
from model_config import VietnameseEmbeddings, load_gemini2, load_gemini15 , load_tiktoken, load_tokenizer
import os
import pandas as pd
from dotenv import load_dotenv
import time
load_dotenv()
def load_final_df():
    final_df_path = r"D:\DATN\QA_System\data_analyze\finaldf.pkl"
    if os.path.exists(final_df_path):
        final_df = pd.read_pickle(final_df_path)
        print(f"Đã load final_df từ file: {final_df_path}")
        return final_df
    else:
        print("File final_df chưa tồn tại.")
        return None



c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
final_df = load_final_df()
documents = convert_df_to_documents(final_df)

In [ ]:
embedding = VietnameseEmbeddings()
#model = load_gemini2()

In [8]:
# Đọc file đầu vào
input_file = r"D:\DATN\QA_System\data_analyze\finaldf0.xlsx"
output_file = "updated_finaldf0_with_questions.xlsx"
data = pd.read_excel(input_file)
# In kiểu dữ liệu và một số giá trị mẫu của cột 'metadata'
print("Kiểu dữ liệu của cột 'metadata':", type(data['metadata'].iloc[0]))
print("Giá trị mẫu trong 'metadata':")
print(data['metadata'].iloc[:5])


Kiểu dữ liệu của cột 'metadata': <class 'str'>
Giá trị mẫu trong 'metadata':
0    {'source': 'Ban giám đốc Đại học.txt', 'id': '0'}
1    {'source': 'Chiến lược phát triển đại học Bách...
2    {'source': 'Chiến lược phát triển đại học Bách...
3    {'source': 'Chiến lược phát triển đại học Bách...
4    {'source': 'Chiến lược phát triển đại học Bách...
Name: metadata, dtype: object


gen cau hoi

In [27]:
import pandas as pd
import json
import logging
import time  # Thêm thư viện time để sử dụng sleep
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Khởi tạo LangChain Chat Model
#llm = ChatOpenAI(model="gpt-4", temperature=0.7)
llm = load_gemini15()

# Prompt để sinh câu hỏi và câu trả lời
template = """
Bạn là chuyên gia trong ngành giáo dục. Nhiệm vụ của bạn là tạo ra các câu hỏi cho phần Câu hỏi thường gặp (FAQs) sắp tới. 
Hãy tạo 1-3 câu hỏi phù hợp cho thông tin được cung cấp với mục đích tạo dataset cho hệ thống RAG. 
Chỉ tạo câu hỏi dựa trên thông tin trong văn bản được cung cấp.

### Quan trọng: Các câu hỏi phải được viết bằng tiếng Việt.

Dưới đây là thông tin:
{context}

Trả lời theo định dạng:
#1: [Câu hỏi]
#2: [Câu hỏi]
...

Nếu không thể tạo câu hỏi, trả lời: "Không thể tạo câu hỏi."
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm | StrOutputParser()

# Đọc file đầu vào
input_file = r"D:\DATN\QA_System\data_analyze\finaldf0.xlsx"
output_file = r"D:\DATN\QA_System\eval\excel2\finaldf0_q.xlsx"
data = pd.read_excel(input_file)

# Sinh câu hỏi cho từng dòng và thêm thời gian chờ
import time

def generate_questions_with_retry(text, retries=5, backoff_factor=2):
    attempt = 0
    while attempt < retries:
        try:
            print("Đang tạo câu hỏi cho đoạn văn...")
            result = chain.invoke({"context": text})  # Gửi nội dung cho mô hình
            print("Câu hỏi đã được tạo thành công.")
            return result.strip()  # Trả về danh sách câu hỏi
        except Exception as e:
            if "ResourceExhausted" in str(e):
                attempt += 1
                wait_time = backoff_factor ** attempt  # Exponential backoff
                print(f"Lỗi khi tạo câu hỏi: {e}. Đang thử lại sau {wait_time} giây...")
                time.sleep(wait_time)  # Đợi trước khi thử lại
            else:
                print(f"Lỗi không xác định: {e}")
                return "Không thể tạo câu hỏi."
    return "Không thể tạo câu hỏi sau nhiều lần thử lại."

# Sử dụng hàm generate_questions_with_retry trong phần xử lý file:
questions_list = []
for index, row in data.iterrows():
    print(f"Đang xử lý dòng {index+1}/{len(data)}...")  # Hiển thị tiến trình
    question = generate_questions_with_retry(row['text'])
    questions_list.append(question)
    
    # Thêm sleep giữa các lần gọi API để không vượt quá giới hạn 10 RPM
    time.sleep(4.1)  # 6 giây để đạt được giới hạn 10 yêu cầu mỗi phút

print("Hoàn thành việc tạo câu hỏi cho tất cả các dòng.")


data['questions'] = questions_list

# Lưu file kết quả
data.to_excel(output_file, index=False, engine="openpyxl")
print(f"File đã được cập nhật và lưu tại {output_file}")


Đang xử lý dòng 1/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử lý dòng 2/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử lý dòng 3/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử lý dòng 4/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử lý dòng 5/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử lý dòng 6/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử lý dòng 7/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử lý dòng 8/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử lý dòng 9/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử lý dòng 10/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử lý dòng 11/981...
Đang tạo câu hỏi cho đoạn văn...
Câu hỏi đã được tạo thành công.
Đang xử 

gen cau tra loi

In [1]:
import os
import sys
code_path = r"D:\DATN\QA_System\code"
sys.path.append(code_path)

# Import trực tiếp từ các file
from utils import convert_df_to_documents
from model_config import VietnameseEmbeddings, load_gemini2, load_gemini15 , load_tiktoken, load_tokenizer
import os
import pandas as pd
from dotenv import load_dotenv
import time
load_dotenv()

c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


True

In [2]:
import pandas as pd
import time
import logging
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Khởi tạo logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Khởi tạo LangChain Chat Model
llm = load_gemini15()  # Giả sử bạn đã có hàm này để tải mô hình

# Prompt để sinh câu trả lời
template = """
Bạn là chuyên gia trong ngành giáo dục. Nhiệm vụ của bạn là tạo ra các câu trả lời cho phần Câu hỏi bên dưới. 
Mục đích để tạo ground truth cho các câu hỏi dựa trên thông tin được cung cấp.
Hãy tạo câu trả lời phù hợp cho các câu hỏi được cung cấp bên dưới. 
### Quan trọng: Các câu trả lời phải được viết bằng tiếng Việt.
Chỉ tạo câu trả lời dựa trên thông tin trong văn bản được cung cấp.

Dưới đây là thông tin:
{context}
Đây là câu hỏi: 
{question}
Trả lời theo định dạng:
#1: [Câu trả lời]
#2: [Câu trả lời]
#3: [Câu trả lời]
...
Câu hỏi và câu trả lời phải cùng định dạng số thứ tự.
Nếu không thể tạo trả lời câu hỏi dựa trên thông tin, trả lời: "Không thể trả lời."
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm | StrOutputParser()

# Đọc file Excel đầu vào
input_file = r"D:\DATN\QA_System\eval\excel2\finaldf0_q_tfidf0.8_clean.xlsx"
output_file = r"D:\DATN\QA_System\eval\excel2\finaldf0_q_tfidf0.8_clean_output.xlsx"
data = pd.read_excel(input_file)

# # Lọc bỏ các dòng có giá trị [] trong cột `filtered_questions`
# data = data[data['filtered_questions'] != '[]']

# Hàm sinh câu trả lời với retry
def generate_answers_with_retry(context, question, retries=5, backoff_factor=2):
    attempt = 0
    while attempt < retries:
        try:
            logging.info(f"Đang tạo câu trả lời cho câu hỏi: {question}")
            result = chain.invoke({"context": context, "question": question})  # Tạo câu trả lời
            return result.strip()
        except Exception as e:
            if "ResourceExhausted" in str(e):
                attempt += 1
                wait_time = backoff_factor ** attempt
                logging.warning(f"Lỗi khi tạo câu trả lời: {e}. Thử lại sau {wait_time} giây...")
                time.sleep(wait_time)
            else:
                logging.error(f"Lỗi không xác định: {e}")
                return "Không thể tạo câu trả lời."
    return "Không thể tạo câu trả lời sau nhiều lần thử lại."

# Sinh câu trả lời cho từng dòng
answers_list = []
for index, row in data.iterrows():
    context = row['text']
    question = row['filtered_questions']
    logging.info(f"Đang xử lý dòng {index+1}/{len(data)}...")
    
    # Sinh câu trả lời với retry
    answer = generate_answers_with_retry(context, question)
    answers_list.append(answer)
    
    # Thêm sleep giữa các lần gọi API để không vượt quá giới hạn tốc độ
    time.sleep(4.1)

# Thêm cột mới `ground_truth` vào DataFrame
data['ground_truth'] = answers_list

# Lưu file kết quả
data.to_excel(output_file, index=False, engine="openpyxl")
logging.info(f"File đã được cập nhật và lưu tại {output_file}")


2024-12-28 07:52:12,353 - INFO - Đang xử lý dòng 1/933...
2024-12-28 07:52:12,354 - INFO - Đang tạo câu trả lời cho câu hỏi: ['#1: Mục tiêu chiến lược phát triển Đại học Bách khoa Hà Nội giai đoạn 2017-2025 là gì?', '#2: Theo chiến lược phát triển, Đại học Bách khoa Hà Nội hướng tới mô hình tổ chức như thế nào?', '#3: Chiến lược phát triển của Đại học Bách khoa Hà Nội giai đoạn 2017-2025 đề cập đến việc xây dựng hình mẫu đại học như thế nào?']
2024-12-28 07:52:19,901 - INFO - Đang xử lý dòng 2/933...
2024-12-28 07:52:19,901 - INFO - Đang tạo câu trả lời cho câu hỏi: ['#1: Mô hình quản trị tiên tiến và hệ thống tài chính vững mạnh đóng vai trò gì trong việc xây dựng hình mẫu thành công và phát triển bền vững của đại học?', '#2: Đại học có kế hoạch nào để nâng cao chất lượng chương trình đào tạo và cải thiện khả năng tìm việc làm của sinh viên tốt nghiệp?', '#3:  Chiến lược nào được sử dụng để thu hút sinh viên ưu tú, học giả xuất sắc trong nước và quốc tế đến học tập, nghiên cứu và làm 

In [13]:
input_file = r"D:\DATN\QA_System\eval\testset\finaldf.xlsx"
data = pd.read_excel(input_file)
# In kiểu dữ liệu và một số giá trị mẫu của cột 'metadata'
print("Kiểu dữ liệu của cột 'metadata':", type(data['metadata'].iloc[0]))
print("Kiểu dữ liệu của cột 'text':", type(data['text'].iloc[0]))
# print("Kiểu dữ liệu của cột 'filtered_questions':", type(data['filtered_questions'].iloc[0]))
# print("Kiểu dữ liệu của cột 'ground_truth':", type(data['ground_truth'].iloc[0]))


Kiểu dữ liệu của cột 'metadata': <class 'str'>
Kiểu dữ liệu của cột 'text': <class 'str'>


In [14]:
# In một số giá trị mẫu của từng cột
print("Giá trị mẫu của cột 'metadata':", data['metadata'].iloc[:5])
print("Giá trị mẫu của cột 'text':", data['text'].iloc[:5])
print("Giá trị mẫu của cột 'filtered_questions':", data['filtered_questions'].iloc[:5])
print("Giá trị mẫu của cột 'ground_truth':", data['ground_truth'].iloc[:5])


Giá trị mẫu của cột 'metadata': 0    {'source': 'Ban giám đốc Đại học.txt', 'id': '0'}
1    {'id': ['0'], 'source': ['Ban giám đốc Đại học...
2    {'source': 'Chiến lược phát triển đại học Bách...
3    {'source': 'Chiến lược phát triển đại học Bách...
4    {'source': 'Chiến lược phát triển đại học Bách...
Name: metadata, dtype: object
Giá trị mẫu của cột 'text': 0    Giám đốc Đại học: PGS.TS. Huỳnh Quyết Thắng\nE...
1    Cụm văn bản này cung cấp thông tin liên hệ của...
2    CHIẾN LƯỢC PHÁT TRIỂN\nTÓM LƯỢC CHIẾN LƯỢC PHÁ...
3    Xây dựng hình mẫu thành công, phát triển bền v...
4    Nâng chất lượng của tất cả các chương trình đà...
Name: text, dtype: object


KeyError: 'filtered_questions'

ghep cap QA 


In [12]:
import pandas as pd
import ast
import logging

# Thiết lập logging để ghi lại các lỗi và cảnh báo
logging.basicConfig(level=logging.WARNING, format="%(asctime)s - %(levelname)s - %(message)s")

# Đọc file đầu vào
input_file = r"D:\DATN\QA_System\eval\excel2\finaldf0_q_tfidf0.8_clean_output.xlsx"
output_file = r"D:\DATN\QA_System\eval\excel2\final_dataset_filtered.xlsx"
data = pd.read_excel(input_file)

# Hàm tách câu hỏi và câu trả lời
def split_questions_and_answers(row):
    try:
        # Chuyển cột 'filtered_questions' thành danh sách
        questions = ast.literal_eval(row['filtered_questions']) if isinstance(row['filtered_questions'], str) else row['filtered_questions']
        
        # Tách cột 'ground_truth' thành danh sách các câu trả lời
        answers = [ans.strip() for ans in row['ground_truth'].split('\n') if ans.startswith('#')]
        
        # Nếu số lượng câu hỏi và câu trả lời không khớp, ghi log cảnh báo
        if len(questions) != len(answers):
            logging.warning(f"Số lượng câu hỏi ({len(questions)}) và câu trả lời ({len(answers)}) không khớp tại dòng {row.name + 1}.")
            # Chỉ giữ lại số lượng khớp tối thiểu
            min_length = min(len(questions), len(answers))
            questions = questions[:min_length]
            answers = answers[:min_length]
        
        # Ghép từng cặp câu hỏi và câu trả lời
        pairs = [{'question': q, 'answer': a} for q, a in zip(questions, answers)]
        return pairs
    except Exception as e:
        logging.error(f"Lỗi khi xử lý dòng {row.name + 1}: {e}")
        return []

# Tạo danh sách các dòng mới từ từng cặp câu hỏi và câu trả lời
new_rows = []
for _, row in data.iterrows():
    pairs = split_questions_and_answers(row)
    for pair in pairs:
        new_rows.append({
            'metadata': row['metadata'],
            'text': row['text'],
            'question': pair['question'],
            'answer': pair['answer']
        })

# Tạo DataFrame mới từ danh sách dòng
new_data = pd.DataFrame(new_rows)

# Lọc bỏ các dòng có chữ "Không thể trả lời" trong cột 'answer'
filtered_data = new_data[~new_data['answer'].str.contains("Không thể trả lời", na=False)]

# Lưu DataFrame đã lọc ra file Excel
filtered_data.to_excel(output_file, index=False, engine='openpyxl')
print(f"File đã được lọc và lưu tại: {output_file}")


2024-12-28 10:37:09,840 - WARNING - Số lượng câu hỏi (3) và câu trả lời (0) không khớp tại dòng 66.
2024-12-28 10:37:09,856 - WARNING - Số lượng câu hỏi (2) và câu trả lời (3) không khớp tại dòng 143.
2024-12-28 10:37:09,924 - WARNING - Số lượng câu hỏi (3) và câu trả lời (0) không khớp tại dòng 910.


File đã được lọc và lưu tại: D:\DATN\QA_System\eval\excel2\final_dataset_filtered.xlsx


dung tf idf de tinh diem cua chunks voi nhung cau hoi da gen

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from pyvi import ViTokenizer
tokenize = ViTokenizer.tokenize

# Đọc file Excel
dataset = pd.read_excel(r"D:\DATN\QA_System\eval\excel2\finaldf0_q.xlsx")
def calculate_tfidf_and_filter(df, threshold=0.1):
    vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words=None)
    
    # Chuẩn bị dữ liệu
    texts = df['text'].tolist()
    all_questions = []
    question_indices = []  # Lưu index bắt đầu của questions cho mỗi text
    start_idx = len(texts)
    
    for _, row in df.iterrows():
        questions = [q.strip() for q in row['questions'].split('\n') if q.strip()]
        question_indices.append((start_idx, start_idx + len(questions)))
        start_idx += len(questions)
        all_questions.extend(questions)
    
    # Tính TF-IDF
    all_docs = texts + all_questions
    tfidf_matrix = vectorizer.fit_transform(all_docs)
    
    scores_list = []
    filtered_questions_list = []
    
    for idx, (text_idx, (q_start, q_end)) in enumerate(zip(range(len(texts)), question_indices)):
        text_vector = tfidf_matrix[text_idx]
        questions = [q.strip() for q in df.iloc[idx]['questions'].split('\n') if q.strip()]
        
        scores = []
        filtered = []
        
        for q_idx, question in enumerate(questions):
            question_vector = tfidf_matrix[q_start + q_idx]
            score = (text_vector * question_vector.T).toarray()[0][0]
            scores.append(score)
            if score > threshold:
                filtered.append(question)
        
        scores_list.append(scores)
        filtered_questions_list.append(filtered)
    
    df['tfidf_scores'] = scores_list
    df['filtered_questions'] = filtered_questions_list
    return df
# Tính toán TF-IDF và lọc câu hỏi
result_df = calculate_tfidf_and_filter(dataset, threshold=0.8)

result_df.to_excel(r"D:\DATN\QA_System\eval\excel2\finaldf0_q_tfidf0.8.xlsx", index=False, engine="openpyxl")


c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


bo nhung dong co cau hoi diem < 0.8

In [2]:
import pandas as pd
import os

# Đọc file Excel
file_path = r"D:\DATN\QA_System\eval\excel2\finaldf0_q_tfidf0.8.xlsx"  # Thay bằng đường dẫn thực tế của file Excel
df = pd.read_excel(file_path)

# Lọc bỏ các dòng có nội dung là [] trong cột 'filtered_questions'
filtered_df = df[df['filtered_questions'] != '[]']

# Tạo tên file mới: thêm "_clean" vào trước phần mở rộng
file_name, file_extension = os.path.splitext(file_path)  # Tách tên file và phần mở rộng
new_file_path = f"{file_name}_clean{file_extension}"

# Lưu file mới
filtered_df.to_excel(new_file_path, index=False)

# In thông báo hoàn thành
print(f"File đã được lưu tại: {new_file_path}")


File đã được lưu tại: D:\DATN\QA_System\eval\excel2\finaldf0_q_tfidf0.8_clean.xlsx


ghi pdf ra txt

In [2]:
import pdfplumber

# Đường dẫn đến file PDF và file TXT
pdf_path = r"D:\DATN\QA_System\eval\KIỂM TRA QUY CHẾ DÀNH CHO K69.pdf"
txt_output_path = r"D:\DATN\QA_System\eval\output_textk69.txt"

# Đọc file PDF và ghi ra file TXT
def extract_text_to_txt(pdf_path, txt_output_path):
    try:
        with pdfplumber.open(pdf_path) as pdf, open(txt_output_path, 'w', encoding='utf-8') as txt_file:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    txt_file.write(text)
                    txt_file.write("\n\n")  # Ngăn cách giữa các trang
        print(f"Đã ghi nội dung từ PDF ra file TXT: {txt_output_path}")
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")

# Gọi hàm xử lý
extract_text_to_txt(pdf_path, txt_output_path)


Đã ghi nội dung từ PDF ra file TXT: D:\DATN\QA_System\eval\output_textk69.txt


loi xuong dong khi dap an la so

In [ ]:
import pandas as pd
import re

# Đường dẫn file TXT và file Excel đầu ra
txt_path = r"D:\DATN\QA_System\eval\output_textk69.txt"
excel_output_path = r"D:\DATN\QA_System\eval\output_textk69.xlsx"

# Hàm kiểm tra nếu dòng là câu hỏi (bắt đầu bằng số và dấu chấm)
def is_question(line):
    return re.match(r"^\d+\.", line.strip())

# Đọc file TXT và chuẩn hóa
def process_txt_to_dataframe(txt_path):
    with open(txt_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    data = []
    current_question = ""
    options = []

    for line in lines:
        line = line.strip()
        if is_question(line):  # Nếu là câu hỏi
            if current_question:  # Nếu đã có câu hỏi trước đó, lưu lại
                full_question = current_question + "\n" + "\n".join(options)
                data.append({"Question": full_question, "Answer": "", "GroundTruth": ""})
                options = []  # Reset danh sách đáp án
            current_question = line  # Lưu câu hỏi mới
        elif line:  # Nếu là dòng đáp án
            options.append(line)

    # Lưu câu hỏi cuối cùng
    if current_question:
        full_question = current_question + "\n" + "\n".join(options)
        data.append({"Question": full_question, "Answer": "", "GroundTruth": ""})

    return pd.DataFrame(data)

# Xử lý và lưu vào file Excel
df = process_txt_to_dataframe(txt_path)
df.to_excel(excel_output_path, index=False, engine="openpyxl")
print(f"File Excel đã được lưu tại: {excel_output_path}")


sau khi xem file pdf va them ground truth thi thu dc dataset quy che sinh vien